
<b><font size="6">Deep Learning Project: Identifying Musical Instruments</font></b>

**Submitted by:**

    Elias Assaf 
    
    Ghada Abu Elzalaf 

<!-- vscode-markdown-toc -->

<!-- vscode-markdown-toc-config
	numbering=true
	autoSave=true
	/vscode-markdown-toc-config -->
<!-- /vscode-markdown-toc -->


# Introduction:

In the field of music information retrieval (MIR), recognizing the musical instruments played within an audio clip is crucial. It enables music search by specific instruments, computes the similarity between compositions and helps recognize musical genres. <br>
In our project we propose 3 different neural network architectures to solve the task, each with it's own properties, CNN's that extract high level features of the input, RNN's that consider temporal properties of the input, and finally using attention models so we can set the model to focus on specific parts of the input, they are trained with single instrument audio clips and tested with polyphonic music with several instruments. <br>
The data is given in a .wav format which are uncompressed waveforms of the signal that come in 2 channels (stereo), in the project we will explore different transformations to apply on the data such as the stft and mel-spectrogram, to get the best possible output results.

<b><font size="4">Environment setup</font></b>

In [ ]:
#requires python version >= 3.8

import torch
torch.multiprocessing.set_sharing_strategy('file_system')

from torchvision import datasets
from torchvision import transforms

import torch.nn as nn
import torch.nn.functional as nnF
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import random
import torchvision.transforms.functional as F

import os
import zipfile
from scipy.io import wavfile
from torch.utils.data import Dataset, DataLoader
import glob

import librosa
import librosa.display

import time

import pytorch_lightning as pl

from tqdm import tqdm

from einops import rearrange
from einops.layers.torch import Rearrange

import gc

import wget

import os

from torch.multiprocessing import Manager

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import pandas as pd

import csv

# !pip install tabulate
from tabulate import tabulate

In [ ]:
#constants
SAMPLING_RATE = 44100
N_FFT = 4096
N_MELS = 224


In [ ]:
# dataset download 

data_path = "./data" 

base_url = "https://zenodo.org/record/1290750/files"

file_list = [
    "IRMAS-TrainingData.zip",
    "IRMAS-TestingData-Part1.zip",
    "IRMAS-TestingData-Part2.zip",
    "IRMAS-TestingData-Part3.zip",
]

# Path to folder with the dataset
dataset_folder = f"{data_path}/irmas"
os.makedirs(dataset_folder, exist_ok=True)

for file in tqdm(file_list):
    url = f"{base_url}/{file}"
    if not os.path.exists(f"{dataset_folder}/{file}"):
        wget.download(url, f"{dataset_folder}/{file}")
for file in tqdm(file_list):
    if not os.path.exists(f"{dataset_folder}/{file.split('.')[0]}"):
        with zipfile.ZipFile(f"{dataset_folder}/{file}", "r") as ziphandler:
            ziphandler.extractall(dataset_folder)
print('done')
trainingSet_folder = f"{dataset_folder}/IRMAS-TrainingData"
testingSet_folder = f"{dataset_folder}/IRMAS-TestingData"


# Dataset and Data Preparation

## Data set

IRMAS dataset which includes musical audio files with metadata files that describe the prominent instrument in each
recording.

https://www.upf.edu/web/mtg/irmas

### Training data

6705 audio files in 16 bit stereo wav format sampled at 44.1kHz. where each file is a 3 second signal from 2000 different recordings.

The instruments are:
- cello - cel(388)
- clarinet - cla(505)
- flute - flu(451)
- guitar - gac(637)
- guitar - gel(760)
- organ - org(682)
- piano - pia(721)
- saxophone - sax(626)
- trumpet - tru(577)
- violin - vio(580)
- human singing voice - voi(778)

where each number inside the parentheses describe the number of audio files that contain that specific instrument.

### Test data

2874 files in 16 bit stereo wav format sampled at 44.1kHz.
Each file ranges from 5-20 seconds long, where each files has 1 or more instrument.

Each music file is accompanied by a text file that has 1 instrument annotation per line.

## Data Set Class
We build a Dataset class to help us extract the data sampels easily and apply transforms on them.

We will describe the functionality and purpose of each transformation later on, as to show illustrative figures along with the explanation.

Note: One of the problems that we had was that applying our chosen transformations took a long time and was a bottleneck in the training process, where each epoch took around 2 mins to finish, to overcome this problem we added an option for caching the results (while also keeping in mind random transformations), we have around 1700 files for the training data where each result after the transform is an image of size about 224x224 or 2x224x224, all in all, this results in around 100 Mb of storage which isn't much and so we can easily keep all the results in ram, to maximize the throughput we also use a parallel dictionary (from multiprocessing package using Manager) so we can reuse the same dataset for multiple models and multiple data loader processes.<br>
All of these optimizations resulted in a training time of about 2-3 seconds per epoch (down from 2 mins) for some of our models (on i5-8600K and GTX 1070 ti).

In [ ]:
class IRMASDataset(Dataset): 
    def __init__(self, wav_dir, split='train', mono=False, cache=False, transform=None, target_transform=None, random_transform=None):
        # the class gets -wav_dir- which is the file path, -split- for categorizing the data i.e. 'train', 'valid' or 'test'
        # -mono- if true gets the mono file else gets the stereo, -cache- if true caches the data after loading, -transform- is the transform 
        # to apply on the audio data, -traget_transform- is the transform to apply on the labels.
        if split == 'train':
            self.paths = glob.glob(wav_dir + "/*/*.wav")
            self.labels = [path.split('/')[4] for path in self.paths]
        elif split == 'valid':
            self.paths = glob.glob(wav_dir + "/*/*.wav")[0::10]
            self.labels = [path.split('/')[4] for path in self.paths]
        elif split == 'test':
            self.paths = glob.glob(wav_dir + "/*.wav")
            label_paths = [path.replace(".wav", ".txt") for path in self.paths]
            translator = str.maketrans({'\t': '', '\n': '', ' ': ''})
            self.labels = [[s.translate(translator) for s in open(path, 'r').readlines()]  for path in label_paths]
        else:
            raise Exception("split must be 'train' or 'test' or 'valid'")
            
        self.mono = mono            
        self.transform = transform
        self.target_transform = target_transform
        self.random_transform = random_transform
        self.cache = cache
        if(cache):
            self.manager = Manager()
            self._dict = self.manager.dict()

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        # if isinstance(idx, slice):
        #     return self.getSlice(idx.start, idx.stop, idx.step)
        
        if self.cache and (to_ret := self._dict.get(idx)): # if cache is true then it checks wether the data in the specified index is cached if not it loads it and caches it.
            return (self.random_transform(to_ret[0]), to_ret[1]) if self.random_transform else to_ret
        
        wave, _ = librosa.load(self.paths[idx], sr=None, mono=self.mono)
        wave = wave / wave.max()
        label = self.labels[idx]
        if self.transform:
            wave = self.transform(wave)
        if self.target_transform:
            label = self.target_transform(label)
        if(self.cache):
            self._dict[idx] = (wave, label)
        if self.random_transform:
            wave = self.random_transform(wave)
        return wave, label
    
    def clear(self):
        if self.cache:
            self._dict.clear()
    

In [ ]:
test_dataset = IRMASDataset(trainingSet_folder)
test_dataset[5][0].shape

## Data Preperation - Transforms

### One-hot Transform

Our task is a classification problem, so one necessary transform is the one-hot representation of the labels to enable the calculation of the loss function. We extract the unique labels and build a transform class (input for target_transform in the dataset class).

In [ ]:
class one_hot_transform(object):
    def __init__(self, path=trainingSet_folder, testing_data = False):
        instruments = [f.split('/')[-1] for f in glob.glob(path + "/*")]
        self.nti = {f:i for i,f in enumerate(instruments)}
        self.itn = {i:f for i,f in enumerate(instruments)}
        self.n = len(instruments)
        self.eye = np.eye(self.n, dtype='float32')
        self.testing_data = testing_data
    
    #check dimensional representation
    def __call__(self, labels):
        
        if(isinstance(labels, list)):
            idxs = [self.nti[label] for label in labels]
        else:
            idxs = self.nti[labels]
        return torch.tensor(idxs, dtype=torch.long) if self.testing_data else self.eye[idxs]
    
    # returns the names of the predected instruments
    def inverse(self, preds):
        if(isinstance(preds, list)):
            names = [self.itn[pred] for pred in preds]
        elif(torch.is_tensor(preds)):
            names = [self.itn[int(pred)] for pred in preds]
        else:
            names = self.itn[preds]
        return names
    
    def __len__(self):
        return self.n
    
    

And obviously sanity checking it:

In [ ]:
one_hot = one_hot_transform()
print("One hot representation of cello and saxophone:\n{}\n\nInverse transform of some random labels: {}\n{}\n".format(one_hot(["cel", "sax"]), [0,2,6], one_hot.inverse([0,2,6])))

### Mono-Stereo Transform

We also add a quick transform that transforms the audio from stereo to mono:

(for the training in the end we added an option for the dataset input to read as mono since we get a higher throughput reading the data from the start as mono)

In [ ]:
class mono_transform(object):
    def __init(self):
        pass
    def __call__(self, wave):
        return librosa.to_mono(wave)

Sanity checks for the data:

In [ ]:
train_data = IRMASDataset(wav_dir=trainingSet_folder, split='train', target_transform = one_hot_transform())

train_data_loader = DataLoader(train_data, batch_size=64, shuffle=True)#, num_workers=1, prefetch_factor=4)

test_data = IRMASDataset(wav_dir=testingSet_folder, split='test', target_transform = one_hot_transform(testing_data=True))
#data loader can only do 1 batch a time because each sample has a different length
test_data_loader = DataLoader(test_data, batch_size=1, shuffle=False)

Stereo vs mono audio sizes:

In [ ]:
mono_trans = mono_transform()
train_iter = iter(train_data_loader)
train_features, train_labels = next(train_iter)
train_feature_example = train_features[0].numpy()
train_feature_example_mono = mono_trans(train_feature_example)
print("Dual channel size: {}".format(train_feature_example.shape))
print("Mono channel size: {}".format(train_feature_example_mono.shape))


In [ ]:
epo = iter(test_data_loader)

How a batch with no transformations looks like:

In [ ]:
for _ in range(10):
    print(next(train_iter)[0].shape)

In [ ]:
print("Batch Example: {}".format(train_features.size()))

librosa.display.waveshow(train_features[0].numpy(), sr=SAMPLING_RATE)
plt.title("a waveform in time visualization")
plt.gcf().set_size_inches(6,2.5)
plt.show()

### Short Time Fourier Transform (STFT)

The Short Time Fourier Transform (STFT) is a way to look at natural audio signals, it is defined as taking the DFT over windows on the signal, using short windows $20-30 ms$ for speech and $\sim 90ms$ for musical signals, we can treat the signals as stationary, while keeping a good spectral resolution that has valuable information on the underlying signal.

In our signals we take the window length to be approximately 93 ms, This corresponds to 4101 samples per window which we round to 4096 for fft effciency.

In [ ]:
stft_example = np.abs(librosa.stft(train_feature_example, n_fft=N_FFT))

print("The stft dimensions for a dual channel audio at 4096 samples per window: {}\nIn our training data there is 130 windows per sample".format(stft_example.shape))
plt.plot(stft_example[0,:,0:4])
plt.gcf().set_size_inches(6,2.5)
plt.title("FFT of multiple windows example")
plt.legend(["window {} fft".format(i) for i in range(4)])
plt.show()


interestingly the data has 0 values at frequencies larger than about 500, that means that most of our data is useless. 

### Spectogram

Another way to show STFT is to use spectrograms, which model each window as a column, and all the FFT of the windows as an image, in other words a time-frequency relation, these are especially useful because they represent sounds as a 2-d image which allow the use of CNN's ;). <br>
We choose to use the melspectrogram because it extracts better features than the normal spectrogram since it converts the frequency axis from linear Hertz scale to a non-linear mel scale, which more closely approximates the way human ears perceive pitch. 

In [ ]:
#Example of mel spectogram
stft_example_mono = np.abs(librosa.stft(train_feature_example_mono, n_fft=N_FFT))
fig, ax = plt.subplots(figsize=(4, 3))
stft_example_mono_db = librosa.amplitude_to_db(stft_example_mono, ref=np.max)
print(stft_example_mono_db.shape)
img = librosa.display.specshow(stft_example_mono_db, x_axis='time', y_axis='log', sr=SAMPLING_RATE)
fig.colorbar(img, ax=ax, format='%+2.0f dB')

ax.set(title='Spectogram')
plt.show()

We show the spectogram for the mono audio as to not show 2 graphs.

In [ ]:
spectrogram_example = librosa.feature.melspectrogram(y=train_feature_example_mono, sr=SAMPLING_RATE, n_fft=N_FFT, n_mels=N_MELS)
fig, ax = plt.subplots(figsize=(4, 3))
spectrogram_example_db = librosa.power_to_db(spectrogram_example, ref=np.max)
img = librosa.display.specshow(spectrogram_example_db, x_axis='time',y_axis='mel', sr=SAMPLING_RATE)
fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set(title='Mel-frequency Spectrogram')
plt.show()
print(spectrogram_example.shape)

We can see that in the mel-frequency spectogram we get a smoother graph that all the the difference in frequency values are equal, for example in the hz scale we can clearly hear the difference between a 200hz to 400hz signal, but we cannot hear the difference in 8000hz and 8200hz, on the other hand in the mel-scale a 200 difference in the low range is equivalent to a 200 difference in a higher range.

We also notice the difference in the shape of the mel-frequency spectogram (224,259) and the regular spectrogram (2049, 130), which sort of compresses the data from the full spectogram to a smaller space, that keeps the important features, and at the same time we get rid of the many zeros in the stft which gives us a lighter network.

We can also control the variable n_mels to change the number of mel-frequencies that we want to calculate, this helps us get spectograms of desired shapes that better fit our network architecture, in the example above we picked this number to be n_mels=224, that is a good value because it can be divided by 2 many times, and we also used a prebuilt network that needed 224x224 input shape.

And so we define the mel-frequency spectogram transform that we can use in our dataset:

In [ ]:
class mel_frequency_transform(object):
    def __init__(self, sr=SAMPLING_RATE, n_fft=N_FFT, n_mels=N_MELS):
        self.sr = SAMPLING_RATE
        self.n_fft = n_fft
        self.n_mels = n_mels
        
    def __call__(self, wave):
        mel_spectrogram = librosa.feature.melspectrogram(y=wave, sr=self.sr, n_fft=self.n_fft, n_mels=self.n_mels)
        return librosa.power_to_db(mel_spectrogram, ref=np.max)[...,1:1+256]
        #we cut the signal to reach length of 256 instead of 259

class spectogram_transform(object):
    def __init__(self, n_fft=N_FFT):
        self.n_fft = n_fft
        
    def __call__(self, wave):
        spectogram = np.abs(librosa.stft(wave, n_fft=N_FFT))
        return librosa.amplitude_to_db(spectogram, ref=np.max)

In [ ]:
#Sanity check
mel_transform = mel_frequency_transform()
mel_example = mel_transform(train_feature_example)
print("stft shape: {}, mel shape: {}, mel size in Kilobytes: {}\n".format(stft_example.shape, mel_example.shape, mel_example.size * mel_example.itemsize//8000))


### Zero Crossing Rate - ZCR

One more metric that can help is the zero crossing rate, that is defined as the rate at which a signal changes from positive to zero to negative or from negative to zero to positive and is given by:
Where is the signal and

is its length.

The ZCR gives us more information about the underlying signal that the neural network can utilize, we can add it as a channel next to the mel_transform.

We replace the last coloumn of the Mel spectogram with the ZCR to add more information to the data. We crop the spectogram such that we get dimentions that are multiplies of 2, which would make it easier for us to lower the dimintionality of the data in the CNN.

**Important:** as we found out experimenting, the ZCR gave worse results overall, so we didn't include it's results, but kept it here so we can discuss it later on.

In [ ]:
zcr_example = librosa.feature.zero_crossing_rate(train_feature_example_mono, frame_length=N_FFT)
plt.plot(3*np.arange(zcr_example.size)/zcr_example.size, zcr_example.T)
plt.xlabel("Time (s)")
plt.ylabel("ZCR")
plt.show()
print(zcr_example.shape)

In [ ]:

class mel_frequency_ZCR_transform(object):
    def __init__(self, sr=SAMPLING_RATE, n_fft=N_FFT, n_mels=N_MELS):
        self.sr = SAMPLING_RATE
        self.n_fft = n_fft
        self.n_mels = n_mels
        
    def __call__(self, wave):
        mel_spectrogram = librosa.feature.melspectrogram(y=wave, sr=self.sr, n_fft=self.n_fft, n_mels=self.n_mels)
        mel = librosa.power_to_db(mel_spectrogram, ref=np.max)[...,:-1,1:1+256]
        zcr = librosa.feature.zero_crossing_rate(wave, frame_length=self.n_fft)[..., 1:1+256]
        ax = 1 if wave.shape[0] == 2 else 0
        return np.concatenate((mel,zcr), axis=ax).astype('float32')

        #we cut the signal to reach length of 256 instead of 259



In [ ]:


#Sanity check
mel_ZCR_transform = mel_frequency_ZCR_transform()
mel_ZCR_example = mel_ZCR_transform(train_feature_example)
mel_ZCR_example_mono = mel_ZCR_transform(train_feature_example_mono)
print("mel+ZCR shape stereo: {}\nmel+ZCR shape mono: {}\n".format(mel_ZCR_example.shape, mel_ZCR_example_mono.shape))




Zero Crossing Rate - ZCR

One more metric that can help is the zero crossing rate, that is defined as the rate at which a signal changes from positive to zero to negative or from negative to zero to positive and is given by:
Where is the signal and

is its length.

The ZCR gives us more information about the underlying signal that the neural network can utilize, we can add it as a channel next to the mel_transform.

We replace the last coloumn of the Mel spectogram with the ZCR to add more information to the data.

# Model architectures:

We propose 3 models to deal with project purpose.

## Model 1

In [ ]:
#TODO



This model is based on 4 layers of CNN followed by 2 fully connected layers, it gets as input mel spectrogram of an audio clip, and classifies the musical instruments that are present in the audio.<br>
Since the model is based on CNN we can easily change the number of input channels, therefore we can feed it mel spectrogram of one or two channels that represent the mono or stereo audio. 
Also, to experiment the model on variant relu activation functions, we added a parameter to the model that determines the activation function i.e relu with a threshold. We tried Relu and LeakyRelu with threshold = 0.3.

The model consists of blocks as follows: 

**Block 1:**<br>
Convolutional layer with $m$ input channels and outputs $n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**Block 2:**<br>
Convolutional layer with $n$ input channels and outputs $2 \cdot n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**Block 3:**<br>
Convolutional layer with $2 \cdot n$ input channels and outputs $4 \cdot n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**Block 4:**<br>
Convolutional layer with $4 \cdot n$ input channels and outputs $8 \cdot n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**Fully connected 0:**<br> 
Fully connected layer followed by relu activation function.

**Fully connected 1:**<br>
Fully connected layer that outputs one-hot vector representing the classification of the played instruments.

We train the model with $m$ input channels which gets 1 or 2 based on the audio channels, i.e. for mono channel $m=1$ and for stereo $m=2$, and $n$ number of hidden channels, where we trained the model once with $n=16$ and another with $n=32$. In addition, we set in_size as 224x256 which represents the size of the mel spectrogram. 

In [ ]:
class SIMPLE_CNN(nn.Module):
    def __init__(self, in_channels, num_hiddens, latent, in_size, relu_type="relu", neg_slope=0.01):
        super(SIMPLE_CNN, self).__init__()
        self.in_channels = in_channels
        self.num_hiddens = num_hiddens
        self.latent = latent
        self.in_size = in_size
        
        # Using different activation functions for comparision, we used rely and leaky relu with negative slope parameter 
        if relu_type == "relu":
            self.relu = nn.ReLU(inplace=True)
        elif relu_type == "leaky_relu":
            self.relu = nn.LeakyReLU(negative_slope=neg_slope, inplace=True)
        else: 
            raise Exception("relu_type must be 'relu' or 'leaky_relu'")
                
        # self.relu = nn.ReLU(inplace=True) if relu_typr == "relu" else nn.LeakyReLU(negative_slope=0.01, inplace=True)
        self.block1 = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=num_hiddens, kernel_size=(3,3), stride=(2,2), padding=1),
                                    nn.BatchNorm2d(num_hiddens),
                                    self.relu)
        self.block2 = nn.Sequential(nn.Conv2d(in_channels=num_hiddens, out_channels=2*num_hiddens, kernel_size=(3,3), stride=(2,2), padding=1),
                                    nn.BatchNorm2d(2*num_hiddens),
                                    self.relu)

        self.block3 = nn.Sequential(nn.Conv2d(in_channels=2*num_hiddens, out_channels=2*2*num_hiddens, kernel_size=(3,3), stride=(2,2), padding=1),
                                    nn.BatchNorm2d(2*2*num_hiddens),
                                    self.relu)
        
        self.block4 = nn.Sequential(nn.Conv2d(in_channels=2*2*num_hiddens, out_channels=2*2*2*num_hiddens, kernel_size=(3,3), stride=(2,2), padding=1),
                                    nn.BatchNorm2d(2*2*2*num_hiddens),
                                    self.relu)

        self.fc0 = nn.Linear(2*2*2*num_hiddens * self.in_size[0]//16 * self.in_size[1]//16, 8*latent)
        self.fc1 = nn.Linear(8*latent, latent)
        
    def forward(self, inputs):
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = x.view(inputs.shape[0], 2*2*2*self.num_hiddens * self.in_size[0]//16 * self.in_size[1]//16)
        x = self.relu(self.fc0(x))
        return self.fc1(x)


In [ ]:
#Sanity Check:
#build a batch of size 2 and run it through the network
test_net = SIMPLE_CNN(1, 16, 11, (224,256))
test_input = transforms.ToTensor()(mel_transform(train_feature_example_mono))
test_input = torch.stack((test_input,test_input))

x = test_net(test_input)
print("input shape: {}, output shape: {}\n".format(test_input.shape, x.shape))


## Model 2

#TODO


This model is based on 4 layeres of CNN followed by a RNN layer and 2 fully connected layers, that gets as input mel spectogram of an audio clip, and classifies the musical instruments that are present in the audio.<br>
Since the model is based on CNN we can easily change the size of the input, therefore we can feed it mel spectrogram of one or two channels that represent the mono or stereo audio. 
Also, to experiment the model on variant relu activation functions, we added a parameter to the model that determines the activation function i.e relu with a threshold. We tried Relu and LeakyRelu with threshold = 0.3.

The model consists of blocks as follows: 

**Block 1:**<br>
Convolutional layer with $m$ input channels and outputs $n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**Block 2:**<br>
Convolutional layer with $n$ input channels and outputs $2 \cdot n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**Block 3:**<br>
Convolutional layer with $2 \cdot n$ input channels and outputs $4 \cdot n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**Block 4:**<br>
Convolutional layer with $4 \cdot n$ input channels and outputs $8 \cdot n$ channels, the kernel size is $3x3$ and stride $2x2$.<br>
Batch normalization is applied and relu is activated.

**RNN layer:**<br> 
RNN layer that outputs $8 \cdot n$ channels that gets as an input the rearranged output of block 4.

**Fully connected 0:**<br> 
Fully connected layer followed by relu activation function.

**Fully connected 1:**<br>
Fully connected layer that outputs one-hot vector representing the classification of the played instruments.

We train the model with $m$ input channels which gets 1 or 2 based on the audio channels, i.e. for mono channel $m=1$ and for stereo $m=2$, and $n$ number of hidden channels, where we trained the model once with $n=16$ and another with $n=32$. In addition, we set in_size as 224x256 which represents the size of the mel spectrogram.

We chose the "time" sequence of the RNN to be the height channel, we didn't test any other channel because we got good results.

In [ ]:
class CNN_RNN(nn.Module):
    def __init__(self, in_channels, num_hiddens, latent, in_size, relu_type="relu", neg_slope=0.01):
        super(CNN_RNN, self).__init__()
        self.in_channels = in_channels
        self.num_hiddens = num_hiddens
        self.latent = latent
        self.in_size = in_size
        
        # Using different activation functions for comparision, we used rely and leaky relu with negative slope parameter 
        if relu_type == "relu":
            self.relu = nn.ReLU(inplace=True)
        elif relu_type == "leaky_relu":
            self.relu = nn.LeakyReLU(negative_slope=neg_slope, inplace=True)
        else: 
            raise Exception("relu_type must be 'relu' or 'leaky_relu'")
            
        self.block1 = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=num_hiddens, kernel_size=(3,3), stride=(2,2), padding=1),
                                    nn.BatchNorm2d(num_hiddens),
                                    self.relu)
        self.block2 = nn.Sequential(nn.Conv2d(in_channels=num_hiddens, out_channels=2*num_hiddens, kernel_size=(3,3), stride=(2,2), padding=1),
                                    nn.BatchNorm2d(2*num_hiddens),
                                    self.relu)

        self.block3 = nn.Sequential(nn.Conv2d(in_channels=2*num_hiddens, out_channels=2*2*num_hiddens, kernel_size=(3,3), stride=(2,2), padding=1),
                                    nn.BatchNorm2d(2*2*num_hiddens),
                                    self.relu)
        
        self.block4 = nn.Sequential(nn.Conv2d(in_channels=2*2*num_hiddens, out_channels=2*2*2*num_hiddens, kernel_size=(3,3), stride=(4,2), padding=1),
                                    nn.BatchNorm2d(2*2*2*num_hiddens),
                                    self.relu)
        self.rearrange = Rearrange('n c l f -> n l (c f)')
        self.gru = nn.GRU(input_size = 2*2*2*num_hiddens*self.in_size[1]//16, hidden_size = 2*2*2*num_hiddens, batch_first = True)
        self.fc0 = nn.Linear(2*2*2*num_hiddens, 8*latent)
        self.fc1 = nn.Linear(8*latent, latent)
        
    def forward(self, inputs):
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.rearrange(x)
        x, _ = self.gru(x)
        x = self.relu(self.fc0(x[:,-1,:]))
        return self.fc1(x)


In [ ]:
#Sanity Check:
#build a batch of size 2 and run it through the network
test_net = CNN_RNN(1, 16, 11, (224,256))
test_input = transforms.ToTensor()(mel_transform(train_feature_example_mono))
test_input = torch.stack((test_input,test_input))

x = test_net(test_input)
print(test_input.shape, x.shape)


For dual channel we dont use ToTensor() transform since mel already gives us the correct dimensions:

In [ ]:
#Sanity Check:
#build a batch of size 3 for stereo and run it through the network
test_net = CNN_RNN(2, 16, 11, (224,256))
test_input = torch.tensor(mel_transform(train_feature_example))
test_input = torch.stack((test_input,test_input,test_input))

x = test_net(test_input)
print("Input: {}, where batch size 3, dual channel audio 2, mel_spectogram: 224,256\nOutput {}, where batch size: 3, labels: 11".format(test_input.shape, x.shape))


## Model 3

#TODO

Model architecture:<br>
We use [CoAtNet](https://arxiv.org/pdf/2106.04803.pdf).<br>
CoAtNet stands for "Context-Aware Attention Network", which is a type of machine learning model that uses attention mechanisms to better process input data. Attention mechanisms allow the model to focus on certain parts of the input data that are most relevant to the task at hand, rather than processing the entire input equally.<br>
The CoAtNet model is typically composed of two main components: an encoder and a decoder. The encoder processes the input data and generates a "context vector" that represents the context of the input. The decoder then uses this context vector, along with the attention mechanism, to generate the final output.

We use [this implementation](https://github.com/chinhsuanwu/coatnet-pytorch), with slight modifications to fit our needs.<br>
It takes as input a 224x224 image and outputs a prediction.
We adapted the data set dimentions to match the CoAtNet, i.e. we used random crop to get spectograms of the size 224x224.  

We use coatnet_0 model which we have modified to get the following parameters:
- image_size - the dimensions of the input image_size.
- in_channels - the number of input channels.
- num_classes - the number of data classes.

Note: the original open-source model gets only num_classes parameter.

In [ ]:
from coatnet import coatnet_0, count_parameters

test_net = coatnet_0(image_size = (224,224), in_channels=1, num_classes=len(one_hot))
test_input = torch.randn(2, 1, 224, 224)
x = test_net(test_input)

print(test_input.shape, x.shape, count_parameters(test_net))


# Testing Data
## Benchmark
First we divide the input signal to multiple windows each with the length of the training data (3 seconds), and we overlap them by 25% (hyperparameter), and we run the divided signal through the network to get a prediction on each segment, we define this transform in the next subsection.

Next, we will define 3 testing criteria for instrument recognition for varying length inputs, each criterion is calculated after predicting the classes:

- Majority vote:<br>
This metric involves selecting the instrument with the highest number of votes, based on a predetermined threshold. <br>
However, when applied to our specific task, it would ignore the presence of accompanying instruments. This is because musical signals typically consist of multiple instruments playing together, with sounds often overlapping in time. In most cases, the accompaniment instruments are weaker in volume compared to lead instruments such as the piano.
In our context, instruments with more votes win depending on a threshold in our case, we choose the candidates with appearances in atleast a third of the predictions.
- S1 - the average:<br>
This metric calculates the average probability of each instrument in the audio clip by taking the average of the sigmoid outputs class-wise (instrument-wise), then applying a threshold to determine the presence of an instrument. The threshold should be selected carefully since higher threshold results in improved precision but decreased recall, while a lower threshold leads to increased recall but decreased precision. <br>
This approach aims to determine the mean probability of each instrument's presence in the audio clip, but it may not necessarily detect all instruments.
- S2 - the normalized sum:<br>
This method first sums the sigmoid outputs for each class, over the entire audio clip. The values are then normalized by dividing them by the maximum value among all classes, resulting in a scale between 0 and 1. Finally, a threshold is applied to determine the presence of each instrument. <br>
This approach is based on the idea that humans perceive the dominant instrument in a relative sense, such that the strongest instrument is always detected and the presence of other instruments is determined based on their relative strength compared to the most active instrument. The normalization step is used to ensure that the relative strength of each instrument is accurately captured.

In [ ]:
#all of these methods inputs are the outputs of neural networks with no activation function
#they return a tensor with indexes of the predicted instruments
import collections

majority_threshold = 3
def majority_vote(pred):
    instruments = pred.max(1).indices
    count = collections.Counter(instruments.tolist())
    return torch.tensor([k for k,v in count.items() if v >= len(instruments)//majority_threshold], dtype=torch.long, device=pred.device)

s1_threshold = 0.1
def s1_vote(pred):
    # 0.02 <= threshold <= 0.18
    #assume pred is the output of the last layer without an activation function
    s1 = pred.sigmoid().sum(0)
    s1 = nnF.threshold(s1, s1_threshold, 0.0)
    s1 = s1.nonzero(as_tuple=True)
    return s1[0]
    
s2_threshold = 0.4
def s2_vote(pred):
    # 0.2 <= threshold <= 0.6
    #assume pred is the output of the last layer without an activation function
    s2 = pred.sigmoid()
    s2 = s2.sum(0)
    s2 = s2.div_(s2.max())
    s2 = nnF.threshold(s2, s2_threshold, 0.0)
    s2 = s2.nonzero(as_tuple=True)
    return s2[0]
    

In [ ]:
test_t = torch.tensor([[0.0, 0.0, 0.1, 0.0],
                       [0.0, 0.4, 0.0, 0.0],
                       [0.0, 0.0, 1.2, 0.0]])
print("majority: {}, s1: {}, s2: {}".format(majority_vote(test_t), s1_vote(test_t), s2_vote(test_t)))


## Test Data Transform

In [ ]:
#This transformation takes data input data of varying length, cuts it up into equally lengthed pieces and overlaps those pieces together
#using this transform yields a batch like tensor for each input with unfixed size, so multiple inputs cant be batched together.

class test_data_cutter_transform(object):
    def __init__(self, overlap_percent):
        self.overlap_per = overlap_percent
        
    def __call__(self, wave):
        mono = 2 if wave.shape[0] == 2 else 1
        leng = wave.shape[-1]
        if leng < 132299:
            npad = [(0, 0)] * wave.ndim
            npad[-1] = (0, 132299-leng)
            wave = np.pad(wave, npad, constant_values=0.0)
            leng = 132299
        parts = int(np.floor((leng+self.overlap_per*132299)/132299))
        windows = np.empty((parts, mono, 132299))
        window_starts = (np.arange(parts, dtype=np.intc)*(132299-self.overlap_per*132299)).astype(int)
        for i in range(parts):
            windows[i] = wave[...,window_starts[i]:window_starts[i]+132299]
        return windows
    

In [ ]:
# Sanity check
#transform = transforms.Compose([mel_frequency_transform()]), \

test_data = IRMASDataset(wav_dir=testingSet_folder, split='test', mono=False, \
                         transform = transforms.Compose([test_data_cutter_transform(0.25), mel_frequency_transform()]),\
                         target_transform = one_hot_transform(testing_data=True))


#data loader can only do 1 batch a time because each sample has a different length
test_data_loader = DataLoader(test_data, batch_size=None, shuffle=False)



In [ ]:
test_iter = iter(test_data_loader)
for _ in range(10):
    x, y = next(test_iter)
    print(x.shape)


# Metrics

Our vote functions take as input the prediction from the networks and output a tensor where each element is an integer that represents a predicted class (instrument), to use this output with our automatic loggers, we need to transform this multilabel output into a multiclass output, then we can easily calculate the Precision, Recall and F1 scores and log them for the testing data.

And so we define the following function:

In [ ]:
from torchmetrics.classification import MulticlassPrecision, MulticlassRecall, MulticlassF1Score

def multilabel_to_multiclass(p):
    #transforms a multi-label input input multi-class output
    #input: p - tensor(dtype=tensor.int) where each element specifies a label number (from {s1,s2,majority}_vote functions)
    #output: pred - tensor of size 11 where pred[i] == i+1 if i in p
    #we add 1 so we can ignore 0 (didn't predict)
    pred = torch.zeros([len(one_hot)], dtype=p.dtype, device=p.device)
    pred[p] = p+1
    return pred
# [0, 4, 11]
# [1, 0, 0, 0, 5, 0, ..., 12]
#example usage:
#metric = MulticlassPrecision(num_classes=len(one_hot)+1, average='micro' )
#x, y = batch
#y_hat = model(x)
#metric(multilabel_to_multiclass(s1_vote(y_hat)), multilabel_to_multiclass(y))


## PyTorch Lightning Class

We will use PyTorch Lightning module to ease the process of training, we will build a generic class that takes a model as input, and trains it with a Dataloader that we will provide.

To implement the class, we first implement the `__init__` function that initializes it, we can do it like a normal `nn.Module` where we implement all the blocks as we saw in class, but we abstracted that using an input that is an `nn.module` so we can use the same class for all our models, and of course the forward function which simply passes the input through the model.

To train and validate the model we implement the `training_step` and `validation_step` functions, which receive a batch and batch_index as input, it simply passes the batch through the model, then calculates the loss function and logs it, the training step returns the loss while validation doesn't return a value, and PyTorch lightning takes care of the boilerplate code.

We also need to add `configure_optimizers` which simply returns an adam optimizer on the parameters of the model.

Thus we can run training on any model using the PyTorch lightning trainer class.

For testing we saw earlier that each sample has a varying length, so the testing step needs to consider the input windows, we did so in the {majority,s1,s2}_vote functions that accept the output of the model on training data as inputs, and outputs the prediction, we then apply a transformation from a multilabel prediction to a multiclass prediction where multiple classes can be present, and calculate metrics such as the F1, Precision and Recall on them

The rest of the functions implemented in the class were used at the start of the project to find out the maximum batch size for a model, but were abandoned after.


In [ ]:
class LModel(pl.LightningModule):
    def __init__(self, model, data_loader=None, valid_loader=None, lr=0.001, batch_size=64):
        super(LModel, self).__init__()
        self.model = model
        self.lr = lr
        self.batch_size = batch_size
        self.data_loader = data_loader
        
        ####################
        
        self.precision_majority_micro = MulticlassPrecision(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.precision_majority_macro = MulticlassPrecision(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        
        self.recall_majority_micro = MulticlassRecall(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.recall_majority_macro = MulticlassRecall(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        
        self.f1score_majority_micro = MulticlassF1Score(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.f1score_majority_macro = MulticlassF1Score(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        ####################
        
        self.precision_s1_micro = MulticlassPrecision(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.precision_s1_macro = MulticlassPrecision(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        
        self.recall_s1_micro = MulticlassRecall(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.recall_s1_macro = MulticlassRecall(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        
        self.f1score_s1_micro = MulticlassF1Score(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.f1score_s1_macro = MulticlassF1Score(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        ####################
        
        self.precision_s2_micro = MulticlassPrecision(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.precision_s2_macro = MulticlassPrecision(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        
        self.recall_s2_micro = MulticlassRecall(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.recall_s2_macro = MulticlassRecall(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
        
        self.f1score_s2_micro = MulticlassF1Score(num_classes=len(one_hot)+1, multidim_average='global', average='micro', ignore_index=0, validate_args=False)
        self.f1score_s2_macro = MulticlassF1Score(num_classes=len(one_hot)+1, multidim_average='global', average='macro', ignore_index=0, validate_args=False)
    
    def forward(self, inputs):
        return self.model(inputs)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nnF.binary_cross_entropy_with_logits(y_hat, y)
        
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nnF.binary_cross_entropy_with_logits(y_hat, y)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        majority = multilabel_to_multiclass(majority_vote(y_hat))
        s1 = multilabel_to_multiclass(s1_vote(y_hat))
        s2 = multilabel_to_multiclass(s2_vote(y_hat))
        y_multiclass = multilabel_to_multiclass(y)
        
        ####################
        
        self.precision_majority_micro(majority, y_multiclass)
        self.log("P_majority_micro", self.precision_majority_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.precision_majority_macro(majority, y_multiclass)
        self.log("P_majority_macro", self.precision_majority_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        
        self.recall_majority_micro(majority, y_multiclass)
        self.log("R_majority_micro", self.recall_majority_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.recall_majority_macro(majority, y_multiclass)
        self.log("R_majority_macro", self.recall_majority_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        
        self.f1score_majority_micro(majority, y_multiclass)
        self.log("F1_majority_micro", self.f1score_majority_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.f1score_majority_macro(majority, y_multiclass)
        self.log("F1_majority_macro", self.f1score_majority_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        ####################
        
        self.precision_s1_micro(s1, y_multiclass)
        self.log("P_s1_micro", self.precision_s1_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.precision_s1_macro(s1, y_multiclass)
        self.log("P_s1_macro", self.precision_s1_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        
        self.recall_s1_micro(s1, y_multiclass)
        self.log("R_s1_micro", self.recall_s1_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.recall_s1_macro(s1, y_multiclass)
        self.log("R_s1_macro", self.recall_s1_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        
        self.f1score_s1_micro(s1, y_multiclass)
        self.log("F1_s1_micro", self.f1score_s1_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.f1score_s1_macro(s1, y_multiclass)
        self.log("F1_s1_macro", self.f1score_s1_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        ####################
        
        self.precision_s2_micro(s2, y_multiclass)
        self.log("P_s2_micro", self.precision_s2_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.precision_s2_macro(s2, y_multiclass)
        self.log("P_s2_macro", self.precision_s2_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        
        self.recall_s2_micro(s2, y_multiclass)
        self.log("R_s2_micro", self.recall_s2_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.recall_s2_macro(s2, y_multiclass)
        self.log("R_s2_macro", self.recall_s2_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        
        self.f1score_s2_micro(s2, y_multiclass)
        self.log("F1_s2_micro", self.f1score_s2_micro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.f1score_s2_macro(s2, y_multiclass)
        self.log("F1_s2_macro", self.f1score_s2_macro, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = self.lr)
    
    def train_dataloader(self):
         return self.data_loader
        
    def valid_dataloader(self):
         return self.data_loader
        

### Training

First, we define a function that fetches the most recently trained model, to allow stopping and resuming the training process, And another that fetches the best model based on validation loss, to use in testing the models.

In [ ]:
#this function gets the latest checkpoint for a model to resume training once we stopped
#if it doesn't find a checkpoints it returns None so the trainer will start anew.
def get_latest_checkpoint(dire):
    try:
        path = os.path.join(glob.glob(os.path.join(dire, "lightning_logs", "version_*"))[-1], "checkpoints", "last.ckpt")
    except:
        return None
    return path if os.path.exists(path) else None

#this function hopefully gets the best model based on validation loss, will throw an error if a model wasn't found.
#it is meant to use in the testing section
def get_latest_checkpoint2(dire):
    path = os.path.join(glob.glob(os.path.join(dire, "lightning_logs", "version_*"))[-1], "checkpoints", "*")
    files = glob.glob(path)
    return (files[1]) if files else None
    
#to save the last training and resume later
mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always reinitialize beofre using


In [ ]:
# test_net = LModel(coatnet_0(image_size = (224,224), in_channels=1, num_classes=len(one_hot)))
# test_input = torch.randn(2, 1, 224, 224)
# x = test_net(test_input)
# print(x.shape, count_parameters(test_net))


In [ ]:
#here we setup the training datasets with caching, it spawns multiple processes but speed up the training by a large margin (over x40 times)
#preferably call dataset.clear() after being done with them to clear the cache and free up the memory.
train_data_mono = IRMASDataset(wav_dir=trainingSet_folder, split='train', cache=True, mono=True, \
                          transform = transforms.Compose([mel_frequency_transform(), transforms.ToTensor()]), \
                          target_transform = one_hot)

valid_data_mono = IRMASDataset(wav_dir=trainingSet_folder, split='valid', cache=True, mono=True, \
                          transform = transforms.Compose([mel_frequency_transform(), transforms.ToTensor()]), \
                          target_transform = one_hot)

train_data_stereo = IRMASDataset(wav_dir=trainingSet_folder, split='train', cache=True, \
                          transform = transforms.Compose([mel_frequency_transform()]), \
                          target_transform = one_hot)

valid_data_stereo = IRMASDataset(wav_dir=trainingSet_folder, split='valid', cache=True, \
                          transform = transforms.Compose([mel_frequency_transform()]), \
                          target_transform = one_hot)


In [ ]:
# dictionary with model names and paths and models, used for testing the data later
models_info = {} 


Next, we will define the models themselves and run the training on them, we stopped training when the validation loss didn't improve for at least 15 epochs, or the model ran for 100 epochs.

Also after we define the model, we insert its name and directory and the lighting torch module of the model into the `models_info` dictionary, so we can run the testing loops on them later.

If the models were already trained and we just want to run the testing on them we only run the cells where we define the models (and insert them into the dictionary), and simply go down and run the testing cells, this gave us incredible flexibility to run a subset of the models if the notebook/pc crashed which it did, many times :).

#### Model 1 - mono

In [ ]:
#del test_net
gc.collect()
num_hiddens = 32
latent = len(one_hot)
in_size = (224,256)
model_1_mono_dir = os.path.join(os.getcwd(), "checkpoints", "model_1_mono")
model_to_train_1 = SIMPLE_CNN(1, num_hiddens, latent, in_size)

train_data_loader_1 = DataLoader(train_data_mono, batch_size=256, shuffle=True, num_workers=2, prefetch_factor=6)

valid_data_loader_1 = DataLoader(valid_data_mono, batch_size=128, num_workers=1, prefetch_factor=6)

model_1_mono = LModel(model_to_train_1)#, train_data_loader_1, valid_data_loader_1) #test here and in fit

models_info["model_1_mono"] = [model_1_mono, model_1_mono_dir]


In [ ]:

mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always put this line before trainer so it saves to the correct place
trainer = pl.Trainer(accelerator="auto", callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=15),mc], \
                     max_epochs=200, default_root_dir=model_1_mono_dir, log_every_n_steps=10)#, auto_scale_batch_size="binsearch")
#trainer.tune(model_1)
trainer.fit(model_1_mono, train_dataloaders=train_data_loader_1, val_dataloaders=valid_data_loader_1, ckpt_path=get_latest_checkpoint(model_1_mono_dir))


#### Model 1 - stereo

In [ ]:
#del test_net
gc.collect()
num_hiddens = 32
latent = len(one_hot)
in_size = (224,256)
model_1_stereo_dir = os.path.join(os.getcwd(), "checkpoints", "model_1_stereo")
model_to_train_1_stereo = SIMPLE_CNN(2, num_hiddens, latent, in_size)

train_data_loader_1_stereo = DataLoader(train_data_stereo, batch_size=256, shuffle=True, num_workers=2, prefetch_factor=6)

valid_data_loader_1_stereo = DataLoader(valid_data_stereo, batch_size=128, num_workers=1, prefetch_factor=6)

model_1_stereo = LModel(model_to_train_1_stereo)#, train_data_loader_1, valid_data_loader_1) #test here and in fit

models_info["model_1_stereo"] = [model_1_stereo, model_1_stereo_dir]


In [ ]:

mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always put this line before trainer so it saves to the correct place
trainer = pl.Trainer(accelerator="auto", callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=15),mc], \
                     max_epochs=200, log_every_n_steps=10, default_root_dir=model_1_stereo_dir)#, auto_scale_batch_size="binsearch")
#trainer.tune(model_1)
trainer.fit(model_1_stereo, train_dataloaders=train_data_loader_1_stereo, val_dataloaders=valid_data_loader_1_stereo, ckpt_path=get_latest_checkpoint(model_1_stereo_dir))


#### Model 2 - mono

In [ ]:
gc.collect()
num_hiddens = 32
latent = len(one_hot)
in_size = (224,256)
model_2_mono_dir = os.path.join(os.getcwd(), "checkpoints", "model_2_mono")
model_to_train_2 = CNN_RNN(1, num_hiddens, latent, in_size)

train_data_loader_2 = DataLoader(train_data_mono, batch_size=256, shuffle=True, num_workers=2, prefetch_factor=8)

valid_data_loader_2 = DataLoader(valid_data_mono, batch_size=128, num_workers=1, prefetch_factor=2)

model_2_mono = LModel(model_to_train_2)#, train_data_loader_2, valid_data_loader_2)

models_info["model_2_mono"] = [model_2_mono, model_2_mono_dir]


In [ ]:

mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always put this line before trainer so it saves to the correct place
trainer = pl.Trainer(accelerator="auto", callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=15),mc], \
                     max_epochs=200, log_every_n_steps=10, default_root_dir=model_2_mono_dir)#, auto_scale_batch_size="binsearch")
#trainer.tune(model_1)
trainer.fit(model_2_mono, train_dataloaders=train_data_loader_2, val_dataloaders=valid_data_loader_2, ckpt_path=get_latest_checkpoint(model_2_mono_dir))


#### Model 2 - mono - leaky ReLu

In [ ]:
gc.collect()
num_hiddens = 32
latent = len(one_hot)
in_size = (224,256)
model_2_mono_lrelu_dir = os.path.join(os.getcwd(), "checkpoints", "model_2_mono_lrelu")
model_to_train_2 = CNN_RNN(1, num_hiddens, latent, in_size, relu_type="leaky_relu", neg_slope=0.3)

train_data_loader_2 = DataLoader(train_data_mono, batch_size=256, shuffle=True, num_workers=2, prefetch_factor=8)

valid_data_loader_2 = DataLoader(valid_data_mono, batch_size=128, num_workers=1, prefetch_factor=2)

model_2_mono_lrelu = LModel(model_to_train_2)#, train_data_loader_2, valid_data_loader_2)

models_info["model_2_mono_lrelu"] = [model_2_mono_lrelu, model_2_mono_lrelu_dir]


In [ ]:

mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always put this line before trainer so it saves to the correct place
trainer = pl.Trainer(accelerator="auto", callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=15),mc], \
                     max_epochs=200, log_every_n_steps=10, default_root_dir=model_2_mono_lrelu_dir)#, auto_scale_batch_size="binsearch")
#trainer.tune(model_1)
trainer.fit(model_2_mono_lrelu, train_dataloaders=train_data_loader_2, val_dataloaders=valid_data_loader_2, ckpt_path=get_latest_checkpoint(model_2_mono_lrelu_dir))


#### Model 2 - stereo:

In [ ]:
gc.collect()
num_hiddens = 32
latent = len(one_hot)
in_size = (224,256)
model_2_stereo_dir = os.path.join(os.getcwd(), "checkpoints", "model_2_stereo")
model_to_train_2 = CNN_RNN(2, num_hiddens, latent, in_size)

train_data_loader_2 = DataLoader(train_data_stereo, batch_size=256, shuffle=True, num_workers=2, prefetch_factor=8)

valid_data_loader_2 = DataLoader(valid_data_stereo, batch_size=128, num_workers=1, prefetch_factor=2)

model_2_stereo = LModel(model_to_train_2)#, train_data_loader_2, valid_data_loader_2)

models_info["model_2_stereo"] = [model_2_stereo, model_2_stereo_dir]


In [ ]:

mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always put this line before trainer so it saves to the correct place
trainer = pl.Trainer(accelerator="auto", callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=15),mc], \
                     max_epochs=200, log_every_n_steps=10, default_root_dir=model_2_stereo_dir)#, auto_scale_batch_size="binsearch")
#trainer.tune(model_1)
trainer.fit(model_2_stereo, train_dataloaders=train_data_loader_2, val_dataloaders=valid_data_loader_2, ckpt_path=get_latest_checkpoint(model_2_stereo_dir))


#### Model 2 - stereo - leaky ReLu:

In [ ]:
gc.collect()
num_hiddens = 32
latent = len(one_hot)
in_size = (224,256)
model_2_stereo_lrelu_dir = os.path.join(os.getcwd(), "checkpoints", "model_2_stereo_lrelu")
model_to_train_2 = CNN_RNN(2, num_hiddens, latent, in_size, relu_type="leaky_relu", neg_slope=0.3)

train_data_loader_2 = DataLoader(train_data_stereo, batch_size=256, shuffle=True, num_workers=2, prefetch_factor=8)

valid_data_loader_2 = DataLoader(valid_data_stereo, batch_size=128, num_workers=1, prefetch_factor=2)

model_2_stereo_lrelu = LModel(model_to_train_2)#, train_data_loader_2, valid_data_loader_2)

models_info["model_2_stereo_lrelu"] = [model_2_stereo_lrelu, model_2_stereo_lrelu_dir]


In [ ]:

mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always put this line before trainer so it saves to the correct place
trainer = pl.Trainer(accelerator="auto", callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=15),mc], \
                     max_epochs=200, log_every_n_steps=10, default_root_dir=model_2_stereo_lrelu_dir)#, auto_scale_batch_size="binsearch")
#trainer.tune(model_1)
trainer.fit(model_2_stereo_lrelu, train_dataloaders=train_data_loader_2, val_dataloaders=valid_data_loader_2, ckpt_path=get_latest_checkpoint(model_2_stereo_lrelu_dir))


#### Model 3 - mono:

In [ ]:
gc.collect()
num_hiddens = 32
latent = len(one_hot)
in_size = (224,224)
model_3_mono_dir = os.path.join(os.getcwd(), "checkpoints", "model_3_mono")
model_to_train_3 = coatnet_0(image_size = in_size, in_channels=1, num_classes=latent)

train_data_3 = IRMASDataset(wav_dir=trainingSet_folder, split='train', mono=True, cache = True, \
                          transform = transforms.Compose([mel_frequency_transform(), transforms.ToTensor(), transforms.CenterCrop(224)]), \
                          target_transform = one_hot) #add randomCrop

valid_data_3 = IRMASDataset(wav_dir=trainingSet_folder, split='valid', mono=True, cache = True, \
                          transform = transforms.Compose([mel_frequency_transform(), transforms.ToTensor(), transforms.CenterCrop(224)]), \
                          target_transform = one_hot) #add randomCrop

train_data_loader_3 = DataLoader(train_data_3, batch_size=32, shuffle=True, num_workers=3, prefetch_factor=4)

valid_data_loader_3 = DataLoader(valid_data_3, batch_size=8, num_workers=1, prefetch_factor=2)

model_3_mono = LModel(model_to_train_3)#, train_data_loader_3, valid_data_loader_3)

models_info["model_3_mono"] = [model_3_mono, model_3_mono_dir]

In [ ]:

mc = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_last=True, save_top_k=2, mode="min") #always put this line before trainer so it saves to the correct place
trainer = pl.Trainer(accelerator="auto", callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=10),mc], \
                     max_epochs=100, default_root_dir=model_3_mono_dir)#, auto_scale_batch_size="binsearch")
#trainer.tune(model_1)
trainer.fit(model_3_mono, train_dataloaders=train_data_loader_3, val_dataloaders=valid_data_loader_3, ckpt_path=get_latest_checkpoint(model_3_mono_dir))

train_data_3.clear()
valid_data_3.clear()

### Testing

In [ ]:
#Here we set caching to true
#preferably clear the training dataloaders before starting to test as to avoid crashing
one_hot_test_data = one_hot_transform(testing_data=True)
#transform = transforms.Compose([mel_frequency_transform()]), \
test_data_mono = IRMASDataset(wav_dir=testingSet_folder, split='test', mono=True, cache=True, \
                         transform = transforms.Compose([test_data_cutter_transform(0.25), mel_frequency_transform(), lambda x: torch.tensor(x, dtype=torch.float)]),\
                         target_transform = one_hot_test_data)
#data loader can only do 1 batch a time because each sample has a different length
test_data_mono_loader = DataLoader(test_data_mono, batch_size=None, shuffle=False)#, num_workers=2, prefetch_factor=4)

test_data_stereo = IRMASDataset(wav_dir=testingSet_folder, split='test', cache=True, \
                          transform = transforms.Compose([test_data_cutter_transform(0.25), mel_frequency_transform(), lambda x: torch.tensor(x, dtype=torch.float)]), \
                          target_transform = one_hot_test_data)

test_data_stereo_loader = DataLoader(test_data_stereo, batch_size=None, shuffle=False)#, num_workers=2, prefetch_factor=4)

test_data_3_mono = IRMASDataset(wav_dir=testingSet_folder, split='test', mono=True, cache=False, \
                          transform = transforms.Compose([test_data_cutter_transform(0.25), mel_frequency_transform(), lambda x: torch.tensor(x, dtype=torch.float), transforms.CenterCrop(224)]), \
                          target_transform = one_hot_test_data) #add randomCrop

test_data_3_mono_loader = DataLoader(test_data_3_mono, batch_size=None, shuffle=False)#, num_workers=2, prefetch_factor=4)


In [ ]:
print(test_data_mono[0][0].dtype)
test_data_stereo[0][0].dtype
test_data_3_mono[1][0].shape

The next cell is used to define the csv file that we save our testing results into, and a function that adds a row to the said csv

In [ ]:
import csv
key_list = ['F1_majority_macro', 'F1_s1_macro', 'F1_s2_macro',\
            'F1_majority_micro', 'F1_s1_micro', 'F1_s2_micro',\
            'P_majority_macro', 'P_s1_macro', 'P_s2_macro',\
            'P_majority_micro', 'P_s1_micro', 'P_s2_micro',\
            'R_majority_macro', 'R_s1_macro', 'R_s2_macro',\
            'R_majority_micro', 'R_s1_micro', 'R_s2_micro']

if not os.path.isfile("results.csv"):
    with open("results.csv", 'w') as f:
        writer = csv.writer(f)
        row = ['model_name', 'majority_threshold', 's1_threshold', 's2_threshold']
        row.extend(key_list)
        writer.writerow(row)
    
def csv_writer(model_name, dic):
    with open("results.csv", 'a') as f:
        writer = csv.writer(f)
        row = [model_name, majority_threshold, s1_threshold, s2_threshold]
        row.extend([dic[key] for key in key_list])
        writer.writerow(row)


In [ ]:
#clear the training datasets so we can cache the testing datasets
train_data_mono.clear()
valid_data_mono.clear()
train_data_stereo.clear()
valid_data_stereo.clear()

Here we will pop elements of the dictionary `models_info` by first popping all "mono" models and then "stereo" and then "model_3", we did it this way to save on precious ram and utilize caching of only one testing set at a time as to avoid crashes.

In [ ]:

model_keys = models_info.keys()
model_keys = sorted(model_keys, key = lambda x: x.split('_')[2])
mono_keys = [key for key in model_keys if "mono" in key and "3" not in key]
stereo_keys = [key for key in model_keys if "stereo" in key]

for name in mono_keys:
    direc = models_info.pop(name)
    trainer = pl.Trainer(accelerator="auto", enable_checkpointing=False)#, \
                     #limit_test_batches=0.1,)#, auto_scale_batch_size="binsearch")
    data_loader = test_data_stereo_loader if "stereo" in name else test_data_3_mono_loader if "model_3" in name else test_data_mono_loader
    metrics_model = trainer.test(direc[0], verbose=False, dataloaders=data_loader, ckpt_path=get_latest_checkpoint2(direc[1]))
    csv_writer(name, metrics_model[0])
    del direc[:]
test_data_mono.clear()
del test_data_mono_loader
gc.collect()

for name in stereo_keys:
    direc = models_info.pop(name)
    trainer = pl.Trainer(accelerator="auto", enable_checkpointing=False)#, \
                     #limit_test_batches=0.1,)#, auto_scale_batch_size="binsearch")
    data_loader = test_data_stereo_loader if "stereo" in name else test_data_3_mono_loader if "model_3" in name else test_data_mono_loader
    metrics_model = trainer.test(direc[0], verbose=False, dataloaders=data_loader, ckpt_path=get_latest_checkpoint2(direc[1]))
    csv_writer(name, metrics_model[0])
    del direc[:]
test_data_stereo.clear()
del test_data_stereo_loader
gc.collect()

while models_info:
    name, direc = models_info.popitem()
    trainer = pl.Trainer(accelerator="auto", enable_checkpointing=False)#, \
                 #limit_test_batches=0.1,)#, auto_scale_batch_size="binsearch")
    #can probably do the next line better
    data_loader = test_data_stereo_loader if "stereo" in name else test_data_3_mono_loader if "model_3" in name else test_data_mono_loader
    metrics_model = trainer.test(direc[0], verbose=False, dataloaders=data_loader, ckpt_path=get_latest_checkpoint2(direc[1]))
    csv_writer(name, metrics_model[0])
    del direc[:]

test_data_mono.clear()
test_data_stereo.clear()
test_data_3_mono.clear()

# Results
To evaluate the performance of our proposed models, we used 3 methods: precision, recall and F1 measure. <br>
**Precision - P:** <br>
$$P = \frac{t_p}{t_p+f_p}$$ 
where $t_p$ is true positive and $f_p$ is false positive.
This measurement is used to measure the proportion of positive identifications that is actually correct. <br>
**Recall - R:** <br> 
$$R = \frac{t_p}{t_p+f_n}$$
where $t_p$ is true positive and $f_n$ is false negative.
This measurement is used to measure the proportion of actual positives that was identified correctly.<br>
**F1 measure:** <br>
$$F1 = \frac{2PR}{P+R}$$
Where P and R are the precision and recall respectively.

Precision and recall offer a trade-off, i.e., one metric comes at the cost of another. More precision involves a harsher critic (classifier) that doubts even the actual positive samples from the dataset, thus reducing the recall score. On the other hand, more recall entails a lax critic that allows any sample that resembles a positive class to pass, which makes border-case negative samples classified as “positive,” thus reducing the precision. Ideally, we want to maximize both precision and recall metrics to obtain the perfect classifier.

We used the F1 measure since it is more reliable because it calculates the overall performance of the models.


We also calculated micro and macro averages:<br>
**Macro:** 
    The macro average precision is the arithmetic mean of all the precision values for the different classes.<br> 
**Micro:**
    The micro average precision is the sum of all true positives divided by the sum of all true positives and false positives.<br>
Where the difference between micro and macro averaging is that macro averaging gives equal weight to each category while micro averaging gives equal weight to each sample.



In [ ]:
#loading the csv and sorting the results and filtering for the relevant information.
results_16hidden = pd.read_csv('results_16_bce.csv', index_col=0)
results_16hidden = results_16hidden.sort_index().filter(regex="_majority|_s1|_s2", axis=1)
results_16hidden = results_16hidden.reindex(sorted(results_16hidden.columns,\
                                            key=lambda x: (x.split('_')[0], x.split('_')[2], x.split('_')[1])), axis=1)

display(results_16hidden.T.tail())

results_32hidden = pd.read_csv('results_32_bce.csv', index_col=0)
results_32hidden = results_32hidden.sort_index().filter(regex="_majority|_s1|_s2", axis=1)
results_32hidden = results_32hidden.reindex(sorted(results_32hidden.columns,\
                                            key=lambda x: (x.split('_')[0], x.split('_')[2], x.split('_')[1])), axis=1)
display(results_32hidden.T.tail())

In [ ]:
#a modular helper function that makes plotting easier

def bar_plotter(results, kind, model_kind=None, show=True, ax=None, hidden=16, mt='1/3', s1t='0.1', s2t='0.4', legend=True, ylim=1):
    #plots the data in a bar graph
    #results is the pandas dataframe that is sorted as we wanted
    #kind is a string literal in ["F1", "R", "P"]
    #model_kind is a regular expression string that is used to filter the model names to compare different models
    #mt -> majority threshold used, can be found in results csv
    #s1t -> s1 threshold used, can be found in results csv
    #s2t -> s2 threshold used, can be found in results csv
    df = results.filter(like=kind+"_").T
    if model_kind:
        df = df.filter(regex=model_kind)
    ax = df.plot(
            kind='bar',
            ax=ax,
            stacked=False,
            title=r'{} Score with $\bf{{{}}}$ hidden layers{}Majority Threshold: {}, S1 Threshold: {}, S2 Threshold: {}'.format(kind, hidden, '\n', mt, s1t, s2t),
            figsize=(12,6),
            zorder=3,
            alpha=0.9)
    ax.set_ylim(0,ylim)
    ax.legend(prop={'size': 8})
    ax.grid(visible=True, color='k', linestyle='--', alpha=0.8, zorder=1, which='both') #ax.grid(axis='y', linestyle='--')
    ax.set_xticklabels(['Majority','S1\nMacro','S2','Majority','S1\nMicro','S2'], rotation='horizontal')
    if not legend:
        ax.get_legend().remove()
    if show:
        plt.show()

Here we will plot the results of all the models:

We can see that model 3 beat the rest by a large margin in all metrics, we can also see that all the precision (P) scores using macro averaging have the same value, we aren't sure why this happened, as we used the exact same inputs to all the metrics, so we will check the performance of the models based on the F1 scoring, for different models.

Observation: S1 metrics gave the best results overall, we think that adjusting the thresholds on the S2 metric can yield the same or even better results, but unfortunately we haven't the time and resources to do it by the deadline.

We note that model 3 already has defined structure and the number of hidden layers doesn't affect it, so it has the same values in both graphs.

In [ ]:
bar_plotter(results_16hidden, "F1", ax=plt.subplot(131), show=False, hidden=16, legend=False)
bar_plotter(results_16hidden, "R", ax=plt.subplot(132), show=False, hidden=16)
bar_plotter(results_16hidden, "P", ax=plt.subplot(133), show=False, hidden=16, legend=False)

plt.gcf().set_size_inches(20,5)
plt.show()

bar_plotter(results_32hidden, "F1", ax=plt.subplot(131), show=False, hidden=32, legend=False)
bar_plotter(results_32hidden, "R", ax=plt.subplot(132), show=False, hidden=32)
bar_plotter(results_32hidden, "P", ax=plt.subplot(133), show=False, hidden=32, legend=False)

plt.gcf().set_size_inches(20,5)
plt.show()

## Ablation Study
To make the study more inclusive we use the results of the F1 score since it evaluates the overall performance of the networks.

### Mono Vs Stereo
Here we plot the mono vs stereo models. 

We can see that stereo models outperformed the mono models when using the S1 metric while it slightly underperforms when using majority and S2 metrics. 

Since we are working with the fact that s1 gave better results for our thresholds we conclude that using stereo gives a better output.

In [ ]:
bar_plotter(results_16hidden, "F1", hidden=16, model_kind=".*[12]_stereo$|.*[12]_mono$", ylim=0.7)

### ReLU vs Leaky ReLU
Next we examine the effect of the activation function, by comparing the results of using ReLU vs Very leaky ReLU (LReLU):

We can see that they have comparable results, but using regular ReLU gave an ever so slightly better results.

In [ ]:
bar_plotter(results_16hidden, "F1", hidden=16, model_kind="model_2_mono|model_2_stereo", ylim=0.7)

### Number of Hidden Layers

We compared all versions of model 2 with different size of hidden layers i.e. 16 and 32.

We can see that model 2 mono gives better F1 score when using 16 hidden layers in the Macro scoring while the results are the opposite in the Micro scoring.

On the other hand the results in model 2 stereo we don't see such behavior, where the model with 16 hidden layres outperformes the 32 in s1 and s2 metrics in Macro scoring while in Micro scoring 
we see mixed behavior.


In [ ]:
# bar_plotter(results_16hidden, "F1", ax=plt.subplot(121), show=False, hidden=16, model_kind="model_2_", ylim=0.7)
# bar_plotter(results_32hidden, "F1", ax=plt.subplot(122), show=False, hidden=32, model_kind="model_2_", ylim=0.7)
# plt.gcf().set_size_inches(15,6)
# plt.plot()
names_16 = np.array([name+"_16" for name in results_16hidden.index.values], dtype=results_16hidden.index.values.dtype)
temp_16 = results_16hidden.copy()
temp_16 = temp_16.set_index(names_16)

names_32 = np.array([name+"_32" for name in results_32hidden.index.values], dtype=results_32hidden.index.values.dtype)
temp_32 = results_32hidden.copy()
temp_32 = temp_32.set_index(names_32)

concat_results = pd.concat([temp_16, temp_32])

bar_plotter(concat_results, "F1", show=False, hidden='16 Vs. 32', model_kind="model_2_(mono|stereo)_[16 32]", ylim=0.7)
# bar_plotter(concat_results, "F1", ax=plt.subplot(122), show=False, hidden='16 Vs. 32', model_kind="model_2_stereo_[16 32]", ylim=0.7)
# plt.gcf().set_size_inches(15,5)
plt.show()

In [ ]:
bar_plotter(results_32hidden, "F1", ax=plt.subplot(131), show=False, hidden=32, model_kind="model_2_", legend=False)
bar_plotter(results_32hidden, "P", ax=plt.subplot(132), show=False, hidden=32, model_kind="model_2_")
bar_plotter(results_32hidden, "R", ax=plt.subplot(133), show=False, hidden=32, model_kind="model_2_", legend=False)
plt.gcf().set_size_inches(20,5)
plt.plot()

### Comparision with reference

In [ ]:
#display(results_32hidden)
results_model_2_lrelu = results_16hidden.loc[["model_1_mono", "model_2_stereo_lrelu", "model_3_mono"]].filter(like="s1")
results_thiers = pd.DataFrame([0.51, 0.65, 0.54, 0.56, 0.52, 0.62], index=results_model_2_lrelu.columns, columns=["Han"]).T
results_model_2_lrelu = pd.concat([results_model_2_lrelu, results_thiers])
display(results_model_2_lrelu)

ax=plt.subplot(121)
results_model_2_lrelu.filter(like="macro").T.plot(
        kind='bar',
        ax=ax,
        stacked=False,
        title="All Scores compared with reference model",
        figsize=(12,6),
        zorder=3,
        alpha=0.9)
ax.set_ylim(0,1)
ax.legend(prop={'size': 8})
ax.grid(color='k', linestyle='--', alpha=0.7, zorder=1) #ax.grid(axis='y', linestyle='--')
ax.set_xticklabels(['F1','P\nMacro','R'], rotation='horizontal')

ax=plt.subplot(122)
results_model_2_lrelu.filter(like="micro").T.plot(
        kind='bar',
        ax=ax,
        stacked=False,
        title="All Scores compared with reference model",
        figsize=(12,6),
        zorder=3,
        alpha=0.9)
ax.set_ylim(0,1)
ax.legend(prop={'size': 8})
ax.grid(color='k', linestyle='--', alpha=0.7, zorder=1) #ax.grid(axis='y', linestyle='--')
ax.set_xticklabels(['F1','P\nMicro','R'], rotation='horizontal')
ax.get_legend().remove()
plt.show()

<font size="3">We added table representation of the results:</font>

In [ ]:
def results_table(results, Measure): #Measure = "F1" or "P" or "R"
    # prepares a table of the results for each measure i.e. F1, P or R 
    ind = results.index.values
    df = pd.DataFrame(results.filter(regex=Measure, axis=1).to_numpy(),
                      index=pd.Index(ind),
                      columns=pd.MultiIndex.from_product([['Macro','Micro'],['Majority','S1','S2']], names=[Measure, 'Metric:']))
    s = df.style.format()
    s.set_table_styles([
        {'selector': 'th.col_heading', 'props': 'text-align: center;'},
        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
        {'selector': 'td', 'props': 'text-align: center;'},
    ], overwrite=False)
    return s

In [ ]:
print("F1 measure results:")
display(results_table(results_16hidden, "F1"))

In [ ]:
print("Precison measure results:")
display(results_table(results_16hidden, "P"))

In [ ]:
print("Recall measure results:")
display(results_table(results_16hidden, "R"))

# Result analysis

We used 3 different models and compared the performance of each of them where we used different parameters and activation functions.

## Comparison to Existing Algorithms
Ours models achieve a slightly worse performance that the reference model in the paper, but using the coatNet architecture we can surpass the models by about 0.05 in F1 scoring, we can conclude that using a tailored network yields a better result, and we believe that the result of the paper can be further improved using RNN's alongside the CNN network, as we saw an improvement from model 1 to model 2 by adding RNN layer after the CNN network and before the fully connected layers.<br>
Using these methods we can achieve comparable performance to attention networks which have orders of magnitude more parameters.

## Effect of Activation Function
The paper shows that suppressing the negative part of the activation rather than making it all zero improves the performance compared to normal ReLU because making the whole negative part zero might cause some initially inactive units to be never active. Moreover, it shows that using leaky ReLU, which has been proved to work well in the image classification task, also benefits the musical instrument identification. <br>
We expected it to also improve our models but unfortunately that wasn't the case, we believe that if we played more with the thresholds of the metrics and tried different negative slope values would have achieved an improvement.

## Effect of Identification Threshold
We leave this as future work, as we didn't build the correct infrastructure to test multiple thresholds efficiently.

## Effect of Channels Number
Based on our research, the majority of the previous models use as input the mono channel, so we wanted to check wether feeding the model stereo audio would result in better recognition since there is more information in two channels compared with single channel. The results were inconclusive, as we got better performance using the s1 metric with stereo, and worse performance using the majority and s2 metrics with stereo as compared with mono.

## Effect of Latent Size (Number of Hidden Layers)
As the efffect of channels number the results were inconclusive, sometimes it impproved the performance while at other times it worsened it.<br>
We conclude that using a smaller number of hidden layers is better since the models become smaller in size, and the performance difference is negligible.

## Effect of ZCR
We conducted an experiment, where we added ZCR information to the input data to each of the models. The ZCR as mentioned above, adds more information about the audio and was concatenated to the Mel-Spectogram replacing the last coloumn. 
The results showed that the performance is poor, we conclude that happened because the values of the ZCR $[0,1]$ are very different from the values of the mel-spectrogram $[-80,0]dB$ and so using convolutional kernels on both kinds of data mashed together gave worse results. In future work we believe that using information such as the ZCR and the Energy of the signals can improve the performance of the models as we are adding more information that the models can leverage, but they need to be treated differently than appending them as channels along side the spectogram, for example passing them through fully connected networks and combining them at fully connected layers phase of our networks.



# Limitations


We had some challenges during this project:
- To begin with the data set, the training data is 3 sec audio clips, while the testing data is 5-20 sec, so we had to find an efficient way to treat and divide the test set such that it fits the architecture which is trained on 3 sec audio clips.
- Another challenge consedring the data set is that the training data audio clips contain one instrument each where the test data contain arbitary number of instruments, which made the task multi-labeled classification so we had to come up with a way to set the threshold for the classification that considers accompanying instruments such as piano that are usualy weaker than lead instruments.
- Training data throughput, since we had a large number of files and we applied an expensive transformation (mel_transform), most of the training time was spent on loading the data and not the actual training, to overcome this we implemented a caching option for the dataset where it caches the data after pre-processing (while considering random transforms) and works with multi-process dataloaders that require no additional configuration. One drawback is that we traded runtime performance with memory consumption, and we needed to clear the cache of the datasets at different phases to avoid filling our ram and crashing.
- We initially planned to test our models on different metric thresholds, but we couldn't due to technical and time limitations.
- The Precision Score with macro averaging gave the same results for all models and all metrics, we aren't sure why or what it means, but as we used a built in function to calculate it we can rule out a bad implementation. 
- The coatNet architecture didn't accept 2 channel inputs so we couldn't test the effect of mono vs stereo on that particular model.



# References

[1] J. J. Bosch, J. Janer, F. Fuhrmann, and P. Herrera, "A Comparison of Sound Segregation Techniques for Predominant Instrument Recognition in Musical Audio Signals," in Proceedings of the International Society for Music Information Retrieval Conference, pp. 559-564, 2012.
https://ismir2012.ismir.net/event/papers/559_ISMIR_2012.pdf 

[2] Y. Han, J. Kim, and K. Lee, "Deep convolutional neural networks for predominant instrument recognition in polyphonic music," IEEE Transactions on Audio, Speech, and Language Processing, , VOL. 14, NO. 8, MAY 2016.
https://arxiv.org/pdf/1605.09507.pdf 

[3] D. Szeliga, P. Tarasiuk, B. Stasiak, and P. S. Szczepaniak, "Musical instrument recognition with a convolutional neural network and staged training," in Procedia Computer Science, vol. 207, pp. 2493-2502, 2022.
https://www.sciencedirect.com/science/article/pii/S1877050922011966